In [2]:
import os
import re
from bs4 import BeautifulSoup

In [4]:
ls

003_0008.html     html_parse.ipynb


In [5]:
file = '003_0008.html'

In [56]:
with open(file,'r') as f:
    lines = f.readlines()
    f.seek(0)
    raw = f.read()

In [53]:
raw[:100]

'<div class="container">\n<div class="book-utils">\n<div id="qrcode"></div>\n<a href="/Jan_Barsceuski/Za'

In [55]:
lines[:20]

['<div class="container">\n',
 '<div class="book-utils">\n',
 '<div id="qrcode"></div>\n',
 '<a href="/Jan_Barsceuski/Zaroslaje_voziera.epub">epub</a>\n',
 '<div>\xa0</div>\n',
 '<div style="vertical-align: middle">\n',
 '<label class="switch">\n',
 '<input id="dictSwitch" type="checkbox"/>\n',
 '<span class="slider round"></span>\n',
 '</label>\n',
 '</div>\n',
 '<div id="nodictLabel">Падключыць<br/>слоўнікі</div>\n',
 '<div id="dictLabel" style="display: none">Націскайце на<br/>словы ў тэксце</div>\n',
 '</div>\n',
 '<!-- HEADER_FIELD Authors: Ян Баршчэўскі -->\n',
 '<!-- HEADER_FIELD CreationYear: 1842 -->\n',
 '<!-- HEADER_FIELD Edition: Ян Баршчэўскі. Выбраныя творы. Менск, МФ «Беларускі кнігазбор», 1998. -->\n',
 '<!-- HEADER_FIELD FirstPublicationYear: 1843 -->\n',
 '<!-- HEADER_FIELD LangOrig: pol -->\n',
 '<!-- HEADER_FIELD Pravapis: A1957 -->\n']

In [128]:
def meta(file): 
    with open(file,'r') as f:
        raw = f.read()
        
    metadict = {}
    ptrn = re.compile('(?<=<!-- HEADER_FIELD ).*(?= -->)')
    meta = re.findall(ptrn, raw)

    if meta: 
        for m in meta:
            metadict[m.split(':')[0]] = m.split(':')[1].lstrip()
            
    return metadict

In [129]:
meta(file)

{'Authors': 'Ян Баршчэўскі',
 'CreationYear': '1842',
 'Edition': 'Ян Баршчэўскі. Выбраныя творы. Менск, МФ «Беларускі кнігазбор», 1998.',
 'FirstPublicationYear': '1843',
 'LangOrig': 'pol',
 'Pravapis': 'A1957',
 'PublicationYear': '1998',
 'SectionAuthor': 'Балады',
 'StyleGenre': 'мастацкі/балада',
 'Title': 'Зарослае возера',
 'Title2': 'Балада з народных паданняў',
 'Translation': 'Уладзімір Мархель'}

In [130]:
meta(file)['Authors']

'Ян Баршчэўскі'

In [162]:
def lines(raw):
    soup = BeautifulSoup(raw,'html.parser')
    text = soup.find('div', {'class': 'POETRY'})
    lines = [l.getText().strip() for l in text.find_all('p')]
    
    return lines

def struct(raw):
    structure = []
    stanza = []

    for i,line in enumerate(getlines(raw)):

        if i == 0 and not line:
            continue
        
        if line:
            stanza.append(line)

        else:
            structure.append(stanza)
            stanza = []

    return structure

In [163]:
struct(raw)

[['Поўняць на захад ад Неўля прастору',
  'Вобразы дзікай прыроды:',
  'Лесам, як ценем, укрытыя горы',
  'І ў берагах цёмных воды.'],
 ['Зелень травы скрозь ды чырвань тут моху',
  'Роўна даліну заслалі;',
  'Ёсць пасяродку яе вады троху,',
  'Што як люстэрка з аддалі.'],
 ['Возерам колісь была роўнядзь тая,',
  'Лодкі рыбацкія мела;',
  'Ветрам узнятая хваля крутая',
  'Ля берагоў тых шумела.'],
 ['Час (ён спаўняе заўжды прызначэнне)',
  'Хвалі пазбавіў свабоды;',
  'Зверху раскінуў з траў дзікіх скляпенне,',
  'Сціснуўшы возера воды.'],
 ['Возера-вязня схавала цямніца,',
  'Толькі ў малое вакенца',
  'Месяца твар час ад часу глядзіцца,',
  "Промень ад зоркі праб'ецца."],
 ['Ціша магільная там ужо будзе,',
  'Толькі ў вячэрнім тумане',
  'Чутны крык качак, пра возера ж людзі',
  'Гэткае помняць паданне:'],
 ['Снег ішоў, зорка згасала пры зорцы,',
  'Вецер і цемра густая,',
  'Возера ўнізе, вёска на горцы,',
  'Люд у карчме ўжо гуляе.'],
 ['Шум за сталом, кожны куфаль трымае,',
  'Хто

In [204]:
newtree = BeautifulSoup('', 'xml')

In [256]:
newtree = BeautifulSoup('', 'xml')

for stanza in struct(raw):
    for l in stanza:
        st = newtree.new_tag('l')
        st.string = l
        newtree.append(st)     
        
newtree

<?xml version="1.0" encoding="utf-8"?>
<l>Поўняць на захад ад Неўля прастору</l><l>Вобразы дзікай прыроды:</l><l>Лесам, як ценем, укрытыя горы</l><l>І ў берагах цёмных воды.</l><l>Зелень травы скрозь ды чырвань тут моху</l><l>Роўна даліну заслалі;</l><l>Ёсць пасяродку яе вады троху,</l><l>Што як люстэрка з аддалі.</l><l>Возерам колісь была роўнядзь тая,</l><l>Лодкі рыбацкія мела;</l><l>Ветрам узнятая хваля крутая</l><l>Ля берагоў тых шумела.</l><l>Час (ён спаўняе заўжды прызначэнне)</l><l>Хвалі пазбавіў свабоды;</l><l>Зверху раскінуў з траў дзікіх скляпенне,</l><l>Сціснуўшы возера воды.</l><l>Возера-вязня схавала цямніца,</l><l>Толькі ў малое вакенца</l><l>Месяца твар час ад часу глядзіцца,</l><l>Промень ад зоркі праб'ецца.</l><l>Ціша магільная там ужо будзе,</l><l>Толькі ў вячэрнім тумане</l><l>Чутны крык качак, пра возера ж людзі</l><l>Гэткае помняць паданне:</l><l>Снег ішоў, зорка згасала пры зорцы,</l><l>Вецер і цемра густая,</l><l>Возера ўнізе, вёска на горцы,</l><l>Люд у карчме ў

In [257]:
for i in newtree:
    print(i)

<l>Поўняць на захад ад Неўля прастору</l>
<l>Вобразы дзікай прыроды:</l>
<l>Лесам, як ценем, укрытыя горы</l>
<l>І ў берагах цёмных воды.</l>
<l>Зелень травы скрозь ды чырвань тут моху</l>
<l>Роўна даліну заслалі;</l>
<l>Ёсць пасяродку яе вады троху,</l>
<l>Што як люстэрка з аддалі.</l>
<l>Возерам колісь была роўнядзь тая,</l>
<l>Лодкі рыбацкія мела;</l>
<l>Ветрам узнятая хваля крутая</l>
<l>Ля берагоў тых шумела.</l>
<l>Час (ён спаўняе заўжды прызначэнне)</l>
<l>Хвалі пазбавіў свабоды;</l>
<l>Зверху раскінуў з траў дзікіх скляпенне,</l>
<l>Сціснуўшы возера воды.</l>
<l>Возера-вязня схавала цямніца,</l>
<l>Толькі ў малое вакенца</l>
<l>Месяца твар час ад часу глядзіцца,</l>
<l>Промень ад зоркі праб'ецца.</l>
<l>Ціша магільная там ужо будзе,</l>
<l>Толькі ў вячэрнім тумане</l>
<l>Чутны крык качак, пра возера ж людзі</l>
<l>Гэткае помняць паданне:</l>
<l>Снег ішоў, зорка згасала пры зорцы,</l>
<l>Вецер і цемра густая,</l>
<l>Возера ўнізе, вёска на горцы,</l>
<l>Люд у карчме ўжо гуляе.</l